Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/. Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например: KEYWORDS = ['python', 'парсинг'] Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
KEYWORDS = ['python', 'парсинг','SSD','Марс']

In [ ]:
res = requests.get('https://habr.com/ru/all/')
soup_ = BeautifulSoup(res.text, 'lxml')

In [ ]:
news = soup_.find_all('a', class_ = "post__title_link")

href_list = []
for new in news:
    href_ = new.get('href')
    href_list.append(href_)

In [ ]:
def check_paper (KEYWORDS,text):
    for key in KEYWORDS:
        if key in str(text):        
            return True

In [ ]:
frame = pd.DataFrame(columns=['Заголовок', 'Дата', 'Ссылка', 'Текст статьи'])

for link in href_list:
    soup_ = BeautifulSoup(requests.get(link).text, 'lxml')
    text = soup_.find('div', class_ = 'post__body post__body_full').text
    check = check_paper (KEYWORDS,text) 
    if check == True:
        heder = soup_.title.text
        date = soup_.find('span', class_ = 'post__time').get('data-time_published')
        df2 = pd.DataFrame({'Заголовок' : [heder], 'Дата' : [date], 'Ссылка' : [link], 'Текст статьи' : [text]})
        frame = pd.concat([frame, df2])
   
    time.sleep(0.3)

frame

Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com] В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки> Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [1]:
EMAIL = ['89066390161@yandex.ru','maksuxa4@yandex.ru','maks@yandex.ru']
import json

In [2]:

Headers = {   
'Accept': "application/json, text/plain, */*",
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Connection': 'keep-alive',
'Content-Length': '32',
'Content-Type': 'application/json;charset=UTF-8',
'Host': 'identityprotection.avast.com',
'Origin': 'https://www.avast.com',
'Referer': 'https://www.avast.com/',
#'sec-ch-ua': "Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87",
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

In [ ]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {'emailAddresses': EMAIL}
#Headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
req = requests.post(URL, data = json.dumps(data), headers = Headers)

req

In [ ]:
type(req.text)
decoded = json.loads(req.text)

decoded

In [ ]:
site_breaches = pd.DataFrame(req.json()['breaches'])
#columns=['Email', 'date','site','description'])
site_breaches = site_breaches.transpose()
site_breaches

In [ ]:
mail_breaches = pd.DataFrame(req.json()['summary'])
dfValues = list(data_breaches['breaches'])
print(dfValues)
list_email = list(data_breaches['breaches'].index)
print(list_email)
mail_breaches

In [ ]:
def proverka_breachId (par):
    index = 0
    for Val in dfValues:        
        if par in Val:        
            return list_email[index]
        index += 1
        
print(proverka_breachId(16659))

In [ ]:
site_breaches['Email'] = site_breaches['breachId'].apply(proverka_breachId)
site_breaches.drop(['breachId', 'recordsCount','statistics'], axis = 1)